# TP Gonimètre 2 : Evaluation des incertitudes

Dans un premier temps, on désire mesurer le pas du réseau utilisé avec le goniomètre et évaluer l'incertitude sur cette mesure.
Dans un second temps, on réalisera une courbe d'étalonnage afin de mesurer des longueurs d'ondes avec le goniomètre.

## Réglage du goniomètre
**Manipulation**
>Régler la lunette autocollimatrice à l'aide du dispositif d’éclairage du réticule et du miroir plan.
>Une fois la lunette réglée, régler le collimateur. Réduire ensuite la largeur de la fente. Elle doit être la plus fine possible, tout en étant observable.

## Mesure du pas du réseau

On se place à l'ordre 1.
On redonne la formule des réseaux au minimum de déviation :
$sin(\frac{D_m}{2})=\frac{\lambda}{2a}$

**Manipulation**
> Réaliser la mesure de la déviation minimale pour les raies exploitables de la lampe de Mercure.
> Déterminer la valeur moyenne du pas du réseau et son incertitude-type à l'aide du script Python qui suit.

Si on note les N observations $x_i$, alors la moyenne expérimentale $x ̅ $ et l’écart-type expérimental $s_x$ valent par définition :

Moyenne expérimentale :		$x ̅=\frac{1}{N} ∑_{ i=1 } ^ {N} x_i$

Ecart-type expérimental	:	$s_x= \sqrt{\frac{1}{N-1} ∑_{ i=1 } ^ {N}(x_i-x ̅)^2} $


L’incertitude-type associée à la moyenne $x ̅ $ de N mesures d’une grandeur X vaut :
$u(x ̅ )=\frac{s_x}{\sqrt{N}}$



In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math

Dm=[12.5+(19/60),19+(1/4),20+(1/3),20+(24/60),16+(25/6),16.5+(22/60),17.5+(1/3),22.5+(1/10)]  #Liste des valeurs de déviations minimales
longueurOnde=[435.8,546.1,577.0,579.1,467.8,480,508.6,643.8]  #Liste des longueurs d'ondes
a=[]

N=len(Dm)

#Calcul de la liste de valeurs de pas de réseau
for i in range(N) :
    pas=longueurOnde[i]*1e-9/(2*math.sin(math.radians(Dm[i])/2))
    a.append(pas)

#Calcul de la valeur moyenne et de l'écart-type
amoy=np.mean(a)
stda = np.std(a,ddof=1)

print('La valeur moyenne du pas du réseau est {:.3f} µm'.format(amoy*1e6))
print("L'incertitude-type sur la mesure est {:.3f} nm".format(stda*1e9/np.sqrt(N)))
print("Le réseau comporte {:.0f} par mm".format(1e-3/amoy))

La valeur moyenne du pas du réseau est 1.677 µm
L'incertitude-type sur la mesure est 39.394 nm
Le réseau comporte 596 par mm


## Courbe d'étalonnage

On redonne la formule des réseaux en incidence normale à l'ordre 1 :
$sin(\theta)=\frac{\lambda}{a}$

**Manipulation**
>Eclairer le réseau en incidence normale en expliquant clairement la méthode. Une fois ce réglage fait, il ne doit plus être modifié. Vous utiliserez la lampe de Mercure et celle de Cadmium.

>A l'aide du script Python, tracer la courbe d'étalonnage du goniomètre et la modéliser par une droite. 

La régression linéaire s’obtient avec la fonction np.polyfit(Xexp,Yexp,1). Le dernier argument désigne l’ordre du polynôme par lequel est faite la régression, toujours égal à 1 pour une régression linéaire. Cette fonction renvoie un tableau à deux éléments qui désignent les coefficients de la régression.

In [3]:
#Données mesurées
theta=np.array([167.5+1/20,171.5+14/120,172.5+1/4,172.5+21/60,168.5+1/4,169+9/60,170+13/60,175+1/4])
theta=theta-152+1/5
longueurOnde=np.array([435.8,546.1,577.0,579.1,467.8,480,508.6,643.8])
pas=amoy

#Calcul de sin(theta)
sintheta=[]

for i in range(len(theta)) :
    sinusRadian=math.sin(math.radians(theta[i]))
    sintheta.append(sinusRadian)

#Utilisatin d'un modèle affine
p=np.polyfit(longueurOnde,sintheta,1)
modele=p[0]*longueurOnde+p[1]
    
plt.close()
plt.figure()
plt.plot(longueurOnde,sintheta,'+')
plt.plot(longueurOnde,modele,'r')
plt.grid()
plt.show()



## Barres d'incertitude

Il faut rajouter sur ce graphique, les barres d'incertitudes (incertitude de type B).
Ici, la source d’incertitude prédominante est celle du repérage de l'angle effectué par l’opérateur : il est donc nécessaire d’estimer un intervalle de valeurs à l’intérieur duquel on est sûr que la grandeur mesurée appartient.

Pour arriver à exprimer l'incertitude de type B sous forme d'un écart-type, on doit recourir à une loi de probabilité. Dans le cas d’une évaluation de type B, l’incertitude-type s’identifie à l’écart-type de la distribution choisie.

Au niveau CPGE, deux cas sont à envisager :
- si on n’a aucune autre information qu’une limite basse et une limite haute pour les valeurs de la grandeur mesurée, alors on suppose que la répartition est uniforme entre ces deux bornes (cas a).
- si on connait la valeur mesurée et l’incertitude-type associée, mais qu’on ne connait pas la distribution sous-jacente, alors on suppose que la distribution est gaussienne (cas b).

Nous nous placerons dans le second cas.

Nous cherchons une incertitude-type composée, c'est à dire l’incertitude-type $u(sin(\theta))$ sur la valeur de la grandeur $sin(\theta)$ calculée à partir de la mesure de $\theta$. Pour notre expérience, on peut estimer l'incertitude-type sur chaque mesure d'angle à 2 minutes d'arc.

### Détermination d’une incertitude-type composée grâce à la méthode Monte Carlo – Simulation d’un processus aléatoire

Nous allons déterminer l'incertitude-type sur chaque valeur de $sin(\theta)$ grâce à des tirages aléatoires avec une probabilité de distribution gaussienne. On utilisera la méthode de Monte Carlo.

La méthode Monte Carlo permet d’étudier la variabilité de la valeur d’une grandeur Y sans utiliser de formule de composition d’incertitudes.
Principe :
- On écrit l’expression de Y en fonction des grandeurs Xi.
- On associe à chaque Xi un intervalle dans lequel sa valeur est supposée appartenir (on suppose les distributions gaussiennes pour simplifier).
- On utilise un code Python pour simuler un tirage au sort des valeurs des grandeurs Xi. on utilise np.random.normal(valeur moyenne, ecart-type de la valeur). Il en résulte une valeur de Y. Cette procédure est répétée un très grand nombre de fois (10000 fois ici) et permet d’obtenir un ensemble de valeurs de Y.
- On peut alors tracer un histogramme des valeurs potentielles de Y.
- Le programme calcule ensuite la valeur moyenne $y ̅ $ et l’incertitude-type associée $u(y ̅)$.


**Manipulation**
>Appliquer la méthode de Monte Carlo pour déterminer l'incertitude-type sur la première valeur de $sin(\theta)$ obtenue.


In [7]:
#Incertitude-type sur théta en degré
deltaTheta=2/60

# Fonction permettant de calculer le sin de théta pour la première valeur de théta mesurée
def sinus(angle):
    return math.sin(math.radians(angle))

# Nombre de tirages aléatoires à effectuer
NN = 10000

# Simulation MC avec une distribution de probabilité uniforme----------------------------------------------------
sintheta1 = [] # initialisation de la liste des valeurs de sin(theta1)

for i in range(0,NN):
    # Tirage aléatoire d'une valeur de theta[1] dans l'intervalle défini précédemment.
    angle = np.random.normal(theta[1],deltaTheta)

    # Création de la liste de valeurs de sin(theta1) résultant
    sintheta1.append(sinus(angle))
    
sintheta1moy = np.mean(sintheta1) # valeur moyenne des valeurs de sin(théta)
usintheta1 = np.std(sintheta1,ddof=1) # écart-type de la distribution des valeurs de sin(théta)
# ddof = 1 permet de calculer l'écart-type de sin(theta1) avec le facteur N - 1.

print("Valeur moyenne de sin(theta1) = {:.2f}".format(sintheta1moy))
print("Incertitude-type de sin(theta1) = {:.4f}".format(usintheta1))


# Tracé de l'histogramme permettant de visualiser la distribution des valeurs de sin(theta) pour la première valeur de théta mesurée
plt.close()
plt.figure()
plt.hist(sintheta1,bins = 'rice')
# Python utilise la règle de Rice pour trouver le nombre d'intervalles de l'histogramme, on peut modifier cette valeur, par exemple bins=15.
plt.title('Résultat du tirage aléatoire des angles après simulation')
plt.xlabel("sin(théta)")
plt.show()


Valeur moyenne de sin(theta1) = 0.34
Incertitude-type de sin(theta1) = 0.0005


On va comparer le résultat obtenu avec celui calculé avec la formule de propagation des incertitudes.

On rappelle que :
$\Delta(sin(\theta))=cos(\theta).\Delta(\theta)$

**Manipulation**
>Calculer les incertitudes-type pour chaque valeur de $sin(\theta)$.
>Conclure.

In [4]:
# Utilisation de la formule de propagation des incertitudes
usintheta1_2=[]

for i in range(len(theta)) :
    incertitude=math.cos(math.radians(theta[i]))*math.radians(deltaTheta)
    usintheta1_2.append(incertitude)

print(usintheta1_2)

[0.0005644951713848599, 0.0005616892082244784, 0.0005497489926762529, 0.000545993108471611, 0.0005456416642333139]


**Conclusion**

>On obtient bien la même inceritude-type avec les deux méthodes.

## Validation du modèle

### Définitions
#### Résidu
Pour une observation donnée, le résidu désigne l’écart algébrique entre la valeur expérimentale obtenue et la valeur prédite par la méthode des moindres carrés (modèle affine obtenu).

Le modèle est adapté si la distribution des résidus est aléatoire autour de la valeur nulle.

#### Ecart normalisé
On définit l’écart normalisé (ou z-score) comme l’écart absolu entre la valeur mesurée x et la valeur de référence $x_{ref}$, divisé par l’incertitude-type :

$z=\frac{\vert x-x_{ref} \vert}{u(x)}$
z est donc le nombre d’incertitudes-types d’écart entre la valeur mesurée et la valeur de référence.

Lorsque z ≤ 2, on considère que le résultat de mesure est compatible avec la valeur de référence.

Lorsque z> 2, on considère qu’il ne l’est pas.

### Principe de la validation du modèle

A un niveau CPGE, la démarche expérimentale doit respecter la procédure d’analyse des résultats expérimentaux suivante :
- Représentation graphique des points expérimentaux avec leurs barres d’incertitude (ici nous ne les représenterons pas).
- Superposition de la courbe représentative du modèle testé.
- Observation graphique des résidus ou des écarts normalisés.
- Conclusion : les données expérimentales sont-elles en accord avec le modèle testé ?
Dans le cas d’une régression linéaire, le modèle testé est une fonction affine ou linéaire.


Pour une observation donnée, le résidu désigne l’écart algébrique entre la valeur expérimentale obtenue et la valeur prédite par la méthode des moindres carrés (équation du modèle).

Le modèle est adapté si la distribution des résidus est aléatoire autour de la valeur nulle.

**Manipulation**
>Représenter les points expérimentaux avec leur barre d'erreur et le modèle affine sur le même graphique
>Représenter ensuite les résidus et les écarts normailsés sur deux autres graphiques.
>
>Conclure sur la validité du modèle.

In [8]:
#incertitude sur sin(theta)
u_sintheta=[0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005]


#Tracé des graphes
plt.close()
plt.figure()
plt.figure(figsize = (10,15))

#Points expérimentaux et modèle
plt.subplot(311)
plt.plot(longueurOnde,sintheta,'+')
plt.plot(longueurOnde,modele,'r',label = "Modèle affine")
plt.errorbar(longueurOnde, sintheta, yerr = u_sintheta, fmt = 'bo',label = "Points expérimentaux")
plt.title("Courbe d'étalonnage")
plt.xlabel(r"$\lambda$ (en nm)")
plt.ylabel(r"$sin(\theta)$")
plt.grid(), plt.legend(loc = 'best')

sinusTheta=np.array(sintheta)
residus = sinusTheta - modele   # calcul des résidus
En = residus/u_sintheta  # calcul des écarts normalisés
# Remarque : il faut faire l’un ou l’autre, mais pas les deux !!!

plt.subplot(312)
#Affichage des résidus
plt.errorbar(longueurOnde, residus, yerr = u_sintheta, fmt = 'bo')
# Pour mieux visualiser la droite correspondant à un résidu nul :
plt.plot([400, 650], [0, 0], 'c--')
plt.xlabel(r"$\lambda$ (en nm)"), plt.xlim(400, 650)
plt.ylabel(r"résidus")
plt.ticklabel_format(axis = 'y', style = 'sci', scilimits = (0,0))
plt.grid()

plt.subplot(313)
# Affichage des écarts normalisés :
plt.plot(longueurOnde, En, 'bo')
# Pour mieux visualiser le domaine des En acceptables :
plt.fill_between([350, 650], y1 = -2, y2 = 2, color = 'cyan', alpha = .1)
plt.xlabel(r"$\lambda$ (en nm)"), plt.xlim(400, 650)
plt.ylabel(r"écarts normalisés")
plt.grid()

plt.show()



## Mesure de longueurs d'onde

A l'aide de la courbe d'étalonnage, Etablir les longueurs d'onde présentes dans la lampe de Sodium. Conclure.

In [40]:
mesuresTheta=np.array([###])

Lambda=[]
for i in range(len(mesuresTheta)):
    longOnde=###
    Lambda.append(longOnde)

print('Les longueurs d''onde (en nm) présentes dans la lampe de sodium sont')
print(Lambda)

Les longueurs donde (en nm) présentes dans la lampe de sodium sont
[437.7918638003932, 495.79547475814326]
